In [1]:
import sys
sys.path.append("..")

from utils.llm import OpenAIClientLLM
from evaluator.prompt_manager import PromptManager
from evaluator.evaluators import KeyPointEvaluator

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
key_point_extract_prompt = """You are an expert at extracting and generating keywords. 
In this task, you will be given a question and a standard answer. 
Based on the standard answer, you need to summarize the key points necessary to answer the question. 
List them as follows:\n1. …\n2. …\nand so on, as needed.\n\n
Example:\nQuestion: What are the significant changes in the newly amended Company Law?\n
Standard Answer: The 2023 amendment to the Company Law introduced several significant changes. 
Firstly, the amendment strengthens the regulation of corporate governance, specifically detailing the responsibilities of the board of directors and the supervisory board【1】. 
Secondly, it introduces mandatory disclosure requirements for Environmental, Social, and Governance (ESG) reports【2】. 
Additionally, the amendment adjusts the corporate capital system, lowering the minimum registered capital requirements【3】. 
Finally, the amendment introduces special support measures for small and medium-sized enterprises to promote their development【4】.
\nKey Points:\n1. The amendment strengthens the regulation of corporate governance, detailing the responsibilities of the board of directors and the supervisory board.\n2. 
It introduces mandatory disclosure requirements for ESG reports.\n3. It adjusts the corporate capital system, lowering the minimum registered capital requirements.\n4. 
It introduces special support measures for small and medium-sized enterprises.\n\n
Question: Comparing the major asset acquisitions of Huaxia Entertainment Co., Ltd. in 2017 and Top Shopping Mall in 2018, which company's acquisition amount was larger?\n
Standard Answer: Huaxia Entertainment Co., Ltd.'s asset acquisition amount in 2017 was larger【1】, amounting to 120 million yuan【2】, whereas Top Shopping Mall's asset acquisition amount in 2018 was 50 million yuan【3】.\n
Key Points:\n1. Huaxia Entertainment Co., Ltd.'s asset acquisition amount in 2017 was larger.\n2. Huaxia Entertainment Co., Ltd.'s asset acquisition amount was 120 million yuan in 2017.\n3. Top Shopping Mall's asset acquisition amount was 50 million yuan in 2018.\n\n
Question: Comparing the timing of sustainability and social responsibility initiatives by Meihome Housekeeping Services Co., Ltd. and Cultural Media Co., Ltd., which company initiated these efforts earlier?\n
Standard Answer: Meihome Housekeeping Services Co., Ltd. initiated its sustainability and social responsibility efforts earlier【1】, in December 2018【2】, whereas Cultural Media Co., Ltd. initiated its efforts in December 2019【3】.\n
Key Points:\n1. Meihome Housekeeping Services Co., Ltd. initiated its sustainability and social responsibility efforts earlier.\n2. Meihome Housekeeping Services Co., Ltd. initiated its efforts in December 2018.\n3. Cultural Media Co., Ltd. initiated its efforts in December 2019.\n\n
Question: Based on the 2017 Environmental and Social Responsibility Report of Green Source Environmental Protection Co., Ltd., how did the company improve community relations through participation in charitable activities, community support and development projects, and public service projects?\n
Standard Answer: Green Source Environmental Protection Co., Ltd. improved community relations through several social responsibility activities. Firstly, in March 2017, the company participated in or funded charitable activities and institutions to support education, health, and poverty alleviation, enhancing the company's social image and brand recognition【1】. 
Secondly, in June 2017, the company invested in the local community, supporting education, health, and social development projects, deepening its connection with the community and promoting overall community well-being and development【2】. 
Finally, in August 2017, the company participated in public service projects such as urban greening and public health improvement projects, enhancing the quality of life in the community and promoting sustainable development【3】. These measures enhanced public perception of the company and improved community relations【4】.\n
Key Points:\n1. In March 2017, the company participated in or funded charitable activities and institutions to support education, health, and poverty alleviation, enhancing the company's social image and brand recognition.\n2. In June 2017, the company invested in the local community, supporting education, health, and social development projects, deepening its connection with the community and promoting overall community well-being and development.\n3. In August 2017, the company participated in public service projects such as urban greening and public health improvement projects, enhancing the quality of life in the community and promoting sustainable development.\n4. These measures enhanced public perception of the company and improved community relations.\n\n
Test Case:\nQuestion: {question}\nStandard Answer: {ground_truth} \n
{formatter}"""

In [4]:
question = "Do I require a silver label?"
ground_truth = """Used manufactured homes (whether de-registered with the BC Manufactured Home Registry or not) may only be offered for sale in BC when they bear an approval label.
A new approval label is required where wiring of a used manufactured home has been completely removed and new wiring has been installed under a permit.
Alterations, including additional wiring to an approved unit must be done under a permit and a new approval label is not required.
When a manufactured home has had additions or outbuildings installed and wired without an electrical installation permit, an electrical contractor must obtain an installation permit, survey the work and submit a declaration confirming that the electrical installation is adequate for the purpose and in good order, in accordance with rule 2-300.
Alternate documentation, such as a letter from the manufacturer indicating that the unit was originally approved, cannot be accepted in place of an approval label. However, if original documentation exists (for example, a specification sheet with CSA number) and there have been no unpermitted alterations to the unit, a silver label may be applied by a safety officer upon notification of an inspection request under a silver label permit and confirmation that the installation is in good order in accordance with rule 2-300. A Used Mobile Home Inspection Report is not required under these circumstances."""

formatter = """Respond ONLY with a JSON object containing:\n
- key_points (list of string)
"Example:\n"
"```json\n"
'{"key_points": ["1. In March 2017, the company participated in or funded charitable activities and institutions to support education, health, and poverty alleviation, enhancing the company's social image and brand recognition.", "2. In June 2017, the company invested in the local community, supporting education, health, and social development projects, deepening its connection with the community and promoting overall community well-being and development.", "3. In August 2017, the company participated in public service projects such as urban greening and public health improvement projects, enhancing the quality of life in the community and promoting sustainable development.", "4. These measures enhanced public perception of the company and improved community relations."]}'
"```" """

In [5]:
raw_key_point = OpenAIClientLLM().generate(key_point_extract_prompt.format(question = question, ground_truth = ground_truth, formatter = formatter))

In [13]:
import json
response_text = raw_key_point.strip().replace('```json', '').replace('```', '')
key_points = json.loads(response_text)['key_points']

In [14]:
key_points

['1. A new approval label is required when the wiring of a used manufactured home has been completely removed and new wiring has been installed under a permit.',
 '2. Alterations, including additional wiring to an approved unit, must be done under a permit and a new approval label is not required.',
 '3. If original documentation exists and there have been no unpermitted alterations to the unit, a silver label may be applied by a safety officer upon notification of an inspection request under a silver label permit.',
 '4. A silver label can be applied if the installation is confirmed to be in good order in accordance with rule 2-300, and no Used Mobile Home Inspection Report is required under these circumstances.']

In [10]:
answer = """Used manufactured homes (whether de-registered with the BC Manufactured Home Registry or not) may only be offered for sale in BC when they bear an approval label.
A new approval label is not required where wiring of a used manufactured home has been completely removed and new wiring has been installed under a permit.
Alternate documentation, such as a letter from the manufacturer indicating that the unit was originally approved, cannot be accepted in place of an approval label. However, if original documentation exists (for example, a specification sheet with CSA number) and there have been no unpermitted alterations to the unit, a silver label may be applied by a safety officer upon notification of an inspection request under a silver label permit and confirmation that the installation is in good order in accordance with rule 2-300. A Used Mobile Home Inspection Report is not required under these circumstances."""

In [15]:
evaluator = KeyPointEvaluator(
    llm=OpenAIClientLLM(),
    prompt_manager=PromptManager()
)

result = evaluator.evaluate(
    answer=answer,
    key_points=key_points
)
result

{'completeness_score': 0.5,
 'irrelevant_score': 0.25,
 'hallucination_score': 0.25,
 'raw_output': {'complete_ids': [3, 4],
  'irrelevant_ids': [2],
  'hallucinate_ids': [1],
  'reasons': 'The generated answer correctly describes the conditions under which a silver label may be applied (key_point 3) and confirms that no Used Mobile Home Inspection Report is required under these circumstances (key_point 4). Key_point 2 is not covered in the generated answer as it discusses alterations including additional wiring, which is not mentioned. Key_point 1 is incorrect because the generated answer actually states that a new approval label is not required when the wiring of a used manufactured home has been completely removed and new wiring has been installed under a permit, contrary to what key_point 1 claims.'}}